In [4]:
from helper import *

def get_random_annulus_graph(n, p1, p2, p_inc):
    
    G1x = random_graph_prob_edges(n, p1)
    G2x = random_graph_prob_edges(100 - n, p2)

    offset = len(G1x.nodes)

    Gx = nx.Graph()

    for node in G1x.nodes:
        Gx.add_node(node)

    for node in G2x.nodes:
        Gx.add_node(node + offset)

    for (u, v) in G1x.edges:
        Gx.add_edge(u, v)

    for (u, v) in G2x.edges:
        Gx.add_edge(u + offset, v + offset)

    #nx.draw(Gx, with_labels=True, font_weight='bold')
    for v2 in G2x.nodes:
        n = 1
        while n > 0:
            v1 = np.random.choice(G1x.nodes)
            Gx.add_edge(v1, v2 + offset)
            if np.random.uniform() > p_inc:
                n -= 1
    return Gx

In [5]:
import time
import numpy as np

from get_hard_in import get_diff

def find_hards(tid):
    global lock, hards, shouldRun, runtime, num_tried, time_start
    
    lock.acquire()
    print("Thread", tid, "started", flush=True)
    
    while shouldRun or time_start - time.time() > 60*60*10:
        
        runtime[tid] = time.time()
        
        lock.release()
        
        try:
            G1x = random_graph_num_edges(100, round(np.random.uniform(200, 2000)) )
            G2x = random_graph_num_edges(100, round(np.random.uniform(200, 2000)) )
            G3x = get_random_annulus_graph(round(np.random.uniform(5, 95)), np.random.uniform(0.1,0.9),np.random.uniform(0.1,0.9),np.random.uniform(0.1,0.5))
            
            gotG4x = False
            c = 0
            while (not gotG4x) and (c < 10):
                c += 1
                seq = np.array([ 2*i for i in range(100) ], dtype='float64')
                np.random.shuffle(seq)
                for i in range(len(seq)):
                    seq[i] + round(np.random.uniform(-i, 0.5*i))

                seq *= np.random.uniform(1000,2000)/sum(seq)

                for i in range(len(seq)):
                    seq[i] = round(seq[i])
                    seq[i] = max(seq[i], 2)

                if (sum(seq) % 2 == 1):
                    seq[0] += 1

                try:
                    G4x = nx.random_degree_sequence_graph(seq)
                    gotG4x = True
                except Exception as e:
                    #print(e, flush=True)
                    pass
            if not gotG4x:
                G4x = random_graph_num_edges(100, round(np.random.uniform(200, 2000)) )
                #print("didnt get g4x")
            
            for Gx in [ G1x, G2x, G3x, G4x ]:
                if not shouldRun:
                    break
                try:
                    diff = get_diff(Gx)
                except Exception as e:
                    print(e, flush=True)
                    continue
                if diff >= 2 or diff == -1:
                    lock.acquire()
                    hards.append((diff, Gx))
                    lock.release()
                lock.acquire()
                num_tried += 1
                print("Tested",num_tried,"graphs, found",len(hards),"hards", flush=True)
                lock.release()
        except Exception as e:
            print(e, flush=True)
            pass
        
        lock.acquire()
    
    print("Thread",tid,"terminated", flush=True)
    lock.release()

In [6]:
import threading
import signal
import numpy as np
import time

from helper import *

###
### START THE MULTITHREADED HARD PROBLEM SEARCH
###

NUM_THREADS = 1

try:
    if threads:
        pass
except:
    hards = []
    threads = []
    runtime = np.array([ 0 for _ in range(NUM_THREADS) ])
    lock = threading.Lock()
    num_tried = 0
    time_start = time.time()

shouldRun = True

for i in range(NUM_THREADS):
    lock.acquire()
    t = threading.Thread(target = find_hards, args=(i, ))
    t.start()
    threads.append(t)
    lock.release()

def term_handler(signum, frame):
    global threads, runtime, lock, shouldRun, num_tried
    
    lock.acquire()
    print("Received interrupt. Threads will resolve shortly", flush=True)
    shouldRun = False
    lock.release()
    for t in threads:
        t.join()
    del threads
    del lock
    del runtime
    
    i = 0
    for (diff, Gx) in hards:
        write_graph_to_file(Gx, "diff = " + str(diff), "./hards/hard" + str(diff) + "_" + str(i) + ".txt")
        i += 1
    
signal.signal(signal.SIGTERM, term_handler)
signal.signal(signal.SIGINT, term_handler)

for t in threads:
    t.join()

Thread 0 started
Graph with 100 nodes and 1618 edges
Received interrupt. Threads will resolve shortly
SAT time: 10.444658279418945
Basic time: 1.263228178024292
Tested 1 graphs, found 1 hards
Thread 0 terminated
